# 1. GPU 사용설정(0 or 1)

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # 1번 GPU만 노출됨

In [ ]:
import json
import os
import glob
import ast  # 문자열을 리스트로 변환하는 모듈

# 데이터 폴더 경로
dataset_root = "data"

# Train & Validation JSON 및 이미지 폴더 경로
train_json_folder = os.path.join(dataset_root, "train/labels")
train_image_folder = os.path.join(dataset_root, "train/images")
val_json_folder = os.path.join(dataset_root, "validation/labels")
val_image_folder = os.path.join(dataset_root, "validation/images")

# YOLO 변환 데이터 저장 폴더
train_output_labels = os.path.join(dataset_root, "train/labels")
val_output_labels = os.path.join(dataset_root, "validation/labels")

# 저장할 폴더 생성
os.makedirs(train_output_labels, exist_ok=True)
os.makedirs(val_output_labels, exist_ok=True)

# **클래스 자동 수집**
unique_classes = set()

# **JSON 파일을 확인하여 모든 클래스 리스트를 자동으로 생성하는 함수**
def collect_classes(json_folder):
    json_files = glob.glob(os.path.join(json_folder, "**", "*.json"), recursive=True)
    for json_file in json_files:
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
        annotations = data.get("Learning Data Info", {}).get("annotations", [])
        for annotation in annotations:
            class_id = annotation.get("class_id")
            unique_classes.add(class_id)  # 클래스 ID 저장

# Train & Validation 폴더에서 클래스 수집
collect_classes(train_json_folder)
collect_classes(val_json_folder)

# **자동으로 YOLO 클래스 매핑 생성**
class_mapping = {class_id: idx for idx, class_id in enumerate(sorted(unique_classes))}
print(f"✅ YOLO 클래스 매핑 완료: {class_mapping}")

# **JSON에서 실제 이미지 크기(해상도)를 가져오는 함수**
def get_image_size_from_json(json_data):
    """JSON 파일에서 이미지 해상도를 가져와 정수형으로 변환"""
    resolution = json_data.get("Raw Data Info", {}).get("resolution", "3840, 2160")
    try:
        width, height = map(int, resolution.split(", "))  # "3840, 2160" → (3840, 2160)
        return width, height
    except ValueError:
        print(f"⚠️ 해상도 값이 잘못됨: {resolution}, 기본값(3840x2160) 사용")
        return 3840, 2160  # 기본 해상도 반환

# **바운딩 박스 좌표 변환 함수**
def parse_bbox(bbox):
    """바운딩 박스 좌표를 리스트로 변환하고 값이 문자열이면 변환"""
    if isinstance(bbox, str):
        try:
            bbox = ast.literal_eval(bbox)  # 문자열을 리스트로 변환
        except (SyntaxError, ValueError):
            print(f"⚠️ 바운딩 박스 좌표 변환 실패: {bbox}")
            return None
    if len(bbox) != 4:
        print(f"⚠️ 바운딩 박스 좌표 개수 오류: {bbox}")
        return None
    return bbox

# JSON 파일을 YOLO 형식으로 변환하는 함수
def convert_json_to_yolo(json_folder, image_folder, output_labels_folder):
    json_files = glob.glob(os.path.join(json_folder, "**", "*.json"), recursive=True)
    print(f"🔍 {json_folder} 내 JSON 파일 개수: {len(json_files)}")

    for json_file in json_files:
        try:
            with open(json_file, "r", encoding="utf-8") as f:
                data = json.load(f)

            # **JSON 파일명에서 확장자 제거하여 이미지 파일명 생성**
            json_filename = os.path.basename(json_file)
            image_filename = os.path.splitext(json_filename)[0] + ".jpg"

            txt_file_path = os.path.join(output_labels_folder, os.path.splitext(json_filename)[0] + ".txt")

            # JSON에서 해상도(이미지 크기) 가져오기
            img_w, img_h = get_image_size_from_json(data)

            annotations = data.get("Learning Data Info", {}).get("annotations", [])
            yolo_labels = []

            for annotation in annotations:
                class_id = annotation.get("class_id")
                bbox = annotation.get("coord")

                if class_id not in class_mapping:
                    print(f"⚠️ 알 수 없는 클래스: {class_id}, 해당 객체는 제외됨.")
                    continue

                yolo_class_id = class_mapping[class_id]

                # **바운딩 박스 좌표 변환**
                bbox = parse_bbox(bbox)
                if bbox is None:
                    continue  # 좌표 변환 실패한 경우 해당 객체 무시

                x_min, y_min, bbox_w, bbox_h = bbox
                center_x = x_min + bbox_w / 2
                center_y = y_min + bbox_h / 2

                # YOLO 정규화 (각 이미지의 실제 해상도 기준)
                center_x /= img_w
                center_y /= img_h
                bbox_w /= img_w
                bbox_h /= img_h

                # **YOLO 형식 데이터 추가 (주석 제거)**
                yolo_label = f"{yolo_class_id} {center_x:.6f} {center_y:.6f} {bbox_w:.6f} {bbox_h:.6f}"
                yolo_labels.append(yolo_label)  # ⚠️ 주석 제거

            with open(txt_file_path, "w") as f:
                f.write("\n".join(yolo_labels))

            print(f"✅ 변환 완료: {txt_file_path} (이미지 크기: {img_w}x{img_h})")

        except Exception as e:
            print(f"⚠️ 변환 실패: {json_file} - {str(e)}")

# Train & Validation 데이터 변환 실행
convert_json_to_yolo(train_json_folder, train_image_folder, train_output_labels)
convert_json_to_yolo(val_json_folder, val_image_folder, val_output_labels)

print("🎯 모든 JSON 파일을 YOLO 포맷으로 변환 완료!")

In [ ]:
# import os
# import glob
# import cv2
# import xml.etree.ElementTree as ET
# import yaml
# from ultralytics import YOLO

# # def convert_xml_to_yolo(xml_path):
# #     """
# #     pklot 폴더의 XML annotation 파일을 YOLO 형식의 txt 파일로 변환합니다.
# #     주차 공간의 occupied 여부를 클래스(Label)로 사용합니다.
# #     """
# #     tree = ET.parse(xml_path)
# #     root = tree.getroot()
    
# #     base = os.path.splitext(os.path.basename(xml_path))[0]
# #     img_path = os.path.join(os.path.dirname(xml_path), base + ".jpg")
# #     if not os.path.exists(img_path):
# #         print(f"이미지 파일이 존재하지 않습니다: {img_path}")
# #         return
    
# #     image = cv2.imread(img_path)
# #     if image is None:
# #         print(f"이미지 로드 실패: {img_path}")
# #         return
# #     img_h, img_w = image.shape[:2]
    
# #     yolo_lines = []
# #     for space in root.findall('space'):
# #         occupied_str = space.get('occupied', '0')
# #         try:
# #             label = int(occupied_str)
# #         except ValueError:
# #             label = 0
        
# #         contour = space.find('contour')
# #         if contour is None:
# #             continue
# #         xs, ys = [], []
# #         for point in contour.findall('point'):
# #             try:
# #                 x = float(point.get('x'))
# #                 y = float(point.get('y'))
# #             except (TypeError, ValueError):
# #                 continue
# #             xs.append(x)
# #             ys.append(y)
# #         if not xs or not ys:
# #             continue
        
# #         xmin, xmax = min(xs), max(xs)
# #         ymin, ymax = min(ys), max(ys)
# #         bbox_w = xmax - xmin
# #         bbox_h = ymax - ymin
# #         center_x = xmin + bbox_w / 2
# #         center_y = ymin + bbox_h / 2
        
# #         # YOLO 형식으로 변환
# #         center_x_norm = center_x / img_w
# #         center_y_norm = center_y / img_h
# #         width_norm = bbox_w / img_w
# #         height_norm = bbox_h / img_h
        
# #         line = f"{label} {center_x_norm:.6f} {center_y_norm:.6f} {width_norm:.6f} {height_norm:.6f}"
# #         yolo_lines.append(line)
    
# #     txt_path = os.path.join(os.path.dirname(xml_path), base + ".txt")
# #     with open(txt_path, 'w') as f:
# #         for line in yolo_lines:
# #             f.write(line + "\n")
# #     print(f"YOLO annotation 파일 생성 완료: {txt_path}")

# # def create_segmented_labels(folder_path):
# #     """
# #     pklotsegmented 폴더 내 모든 jpg 이미지의 상위 폴더(Empty/Occupied)를 참고하여
# #     YOLO 형식의 annotation을 생성합니다.
# #     """
# #     image_files = glob.glob(os.path.join(folder_path, "**/*.jpg"), recursive=True)
# #     for img_file in image_files:
# #         parent_folder = os.path.basename(os.path.dirname(img_file))
# #         label = 1 if parent_folder.lower() == "occupied" else 0
        
# #         annotation_line = f"{label} 0.500000 0.500000 1.000000 1.000000"
# #         txt_file = os.path.splitext(img_file)[0] + ".txt"
# #         if not os.path.exists(txt_file):
# #             with open(txt_file, 'w') as f:
# #                 f.write(annotation_line + "\n")
# #             print(f"Segmented annotation 파일 생성 완료: {txt_file}")

# # 1️⃣ **1단계: 전체 주차장 이미지 학습**
# pklot_folder = "PKLot"
# # xml_files = glob.glob(os.path.join(pklot_folder, "**/**/*.xml"), recursive=True)
# # for xml_file in xml_files:
# #     convert_xml_to_yolo(xml_file)

# # 1단계 데이터 구성 YAML 파일 생성
# data_config_1 = {
#     "train": pklot_folder,
#     "nc": 2,
#     "names": ["empty", "occupied"]
# }

# yaml_file_1 = "pklot_stage1.yaml"
# with open(yaml_file_1, "w") as f:
#     yaml.dump(data_config_1, f, default_flow_style=False)
# print(f"1단계 YAML 파일 생성 완료: {yaml_file_1}")

# # YOLO 모델 학습 (1단계)
# model = YOLO("yolo11n.pt")
# model.train(data=yaml_file_1, epochs=50, batch=16, imgsz=640)
# model.export(format="pt")  # 모델 저장

# # 2️⃣ **2단계: 개별 주차 공간 이미지 학습 (Fine-Tuning)**
# pklotsegmented_folder = "PKLotSegmented"
# create_segmented_labels(pklotsegmented_folder)

# # 2단계 데이터 구성 YAML 파일 생성
# data_config_2 = {
#     "train": pklotsegmented_folder,
#     "val": pklotsegmented_folder,
#     "nc": 2,
#     "names": ["empty", "occupied"]
# }

# yaml_file_2 = "pklot_stage2.yaml"
# with open(yaml_file_2, "w") as f:
#     yaml.dump(data_config_2, f, default_flow_style=False)
# print(f"2단계 YAML 파일 생성 완료: {yaml_file_2}")

# # YOLO 모델 Fine-Tuning (2단계)
# fine_tuned_model = YOLO("yolo11n.pt")  # 1단계 학습 모델을 로드하여 Fine-Tuning
# fine_tuned_model.train(data=yaml_file_2, epochs=10, batch=16, imgsz=640)
# fine_tuned_model.export(format="pt")  # 최종 모델 저장


In [ ]:
# import os
# import glob
# import cv2
# import xml.etree.ElementTree as ET
# import yaml
# from ultralytics import YOLO

# def convert_xml_to_yolo(xml_path):
#     """
#     pklot 폴더 내의 XML annotation 파일을 읽어, 동일 폴더에 YOLO 형식의 txt 파일로 변환합니다.
#     XML 파일과 동일한 이름의 jpg 이미지가 있다고 가정합니다.
#     각 <space> 태그의 'occupied' 속성을 클래스 레이블로 사용합니다.
#     """
#     tree = ET.parse(xml_path)
#     root = tree.getroot()
    
#     # XML 파일과 동일한 이름 (확장자만 jpg)인 이미지 경로 생성
#     base = os.path.splitext(os.path.basename(xml_path))[0]
#     img_path = os.path.join(os.path.dirname(xml_path), base + ".jpg")
#     if not os.path.exists(img_path):
#         print(f"이미지 파일이 존재하지 않습니다: {img_path}")
#         return
    
#     image = cv2.imread(img_path)
#     if image is None:
#         print(f"이미지 로드 실패: {img_path}")
#         return
#     img_h, img_w = image.shape[:2]
    
#     yolo_lines = []
#     for space in root.findall('space'):
#         # 'occupied' 속성을 읽어 클래스 레이블로 사용 (예: 0: empty, 1: occupied)
#         occupied_str = space.get('occupied', '0')
#         try:
#             label = int(occupied_str)
#         except ValueError:
#             label = 0
        
#         contour = space.find('contour')
#         if contour is None:
#             continue
#         xs, ys = [], []
#         for point in contour.findall('point'):
#             try:
#                 x = float(point.get('x'))
#                 y = float(point.get('y'))
#             except (TypeError, ValueError):
#                 continue
#             xs.append(x)
#             ys.append(y)
#         if not xs or not ys:
#             continue
        
#         xmin, xmax = min(xs), max(xs)
#         ymin, ymax = min(ys), max(ys)
#         bbox_w = xmax - xmin
#         bbox_h = ymax - ymin
#         center_x = xmin + bbox_w / 2
#         center_y = ymin + bbox_h / 2
        
#         # 정규화된 좌표 (YOLO 형식: [클래스, center_x, center_y, width, height])
#         center_x_norm = center_x / img_w
#         center_y_norm = center_y / img_h
#         width_norm = bbox_w / img_w
#         height_norm = bbox_h / img_h
        
#         line = f"{label} {center_x_norm:.6f} {center_y_norm:.6f} {width_norm:.6f} {height_norm:.6f}"
#         yolo_lines.append(line)
    
#     txt_path = os.path.join(os.path.dirname(xml_path), base + ".txt")
#     with open(txt_path, 'w') as f:
#         for line in yolo_lines:
#             f.write(line + "\n")
#     print(f"생성된 YOLO annotation 파일: {txt_path}")

# def create_segmented_labels(folder_path):
#     """
#     pklotsegmented 폴더 내의 모든 jpg 이미지에 대해, 
#     상위 폴더 이름(Empty 또는 Occupied)에 따라 annotation 파일을 생성합니다.
    
#     - 상위 폴더가 'occupied'(대소문자 무관)이면 클래스 1 (occupied)
#     - 그 외는 클래스 0 (empty)
    
#     annotation은 이미지 전체를 대상으로 하므로, 정규화된 bounding box는
#     center=(0.5, 0.5), width=1, height=1 로 생성합니다.
#     """
#     image_files = glob.glob(os.path.join(folder_path, "**/**/**/**/*.jpg"), recursive=True)
#     for img_file in image_files:
#         parent_folder = os.path.basename(os.path.dirname(img_file))
#         if parent_folder.lower() == "occupied":
#             label = 1
#         else:
#             label = 0
        
#         annotation_line = f"{label} 0.500000 0.500000 1.000000 1.000000"
#         txt_file = os.path.splitext(img_file)[0] + ".txt"
#         if not os.path.exists(txt_file):
#             with open(txt_file, 'w') as f:
#                 f.write(annotation_line + "\n")
#             print(f"생성된 segmented annotation 파일: {txt_file}")

# # 1. pklot 폴더 내의 모든 XML 파일을 glob 패턴으로 찾아 YOLO annotation으로 변환 (학습 데이터)
# xml_files = glob.glob("PKLotSegmented/**/**/**/*.xml", recursive=True)
# for xml_file in xml_files:
#     convert_xml_to_yolo(xml_file)

# # 2. pklotsegmented 폴더 내의 이미지에 대해, 상위 폴더(Empty/Occupied)에 따라 annotation 생성 (검증 데이터)
# #    여기서는 하위 폴더 구조에 관계없이 폴더 내 모든 jpg 이미지를 처리합니다.
# create_segmented_labels("PKLot")

# # 3. 데이터 구성 YAML 파일 생성
# #    학습 데이터: pklot 폴더, 검증 데이터: pklotsegmented 폴더
# data_config = {
#     "train": "PKLotSegmented",
#     "val": "PKLot",
#     "nc": 2,  # 두 클래스: 0 (empty), 1 (occupied)
#     "names": ["empty", "occupied"]
# }

# yaml_file = "pklot.yaml"
# # with open(yaml_file, "w") as f:
# #     yaml.dump(data_config, f, default_flow_style=False)
# # print(f"데이터 구성 YAML 파일 생성 완료: {yaml_file}")

# # 4. YOLO11n 모델 학습 시작
# model = YOLO("yolo11n.pt")
# model.train(data = yaml_file, epochs=5, batch=8, imgsz=640)

In [ ]:
import os
import xml.etree.ElementTree as ET

# 🚗 클래스 매핑
class_mapping = {
    "승용/승합": 0,
    "버스": 1,
    "화물/기타": 2,
    "이륜차": 3,
    "자전거": 4
}

def convert_xml_to_yolo(xml_path, output_dir):
    """
    XML을 YOLO 포맷으로 변환하는 함수.
    - image 태그 별로 개별 txt 파일을 생성
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # XML 파일명에서 공통 이름 추출 
    base_name = os.path.splitext(os.path.basename(xml_path))[0]

    for image in root.findall("image"):
        # 각 이미지 파일명을 XML에서 추출
        img_id = int(image.get("id"))  
        img_name = f"{base_name}_{img_id:03d}.jpg"  

        img_width = int(image.get("width"))
        img_height = int(image.get("height"))

        yolo_annotations = []

        for box in image.findall("box"):
            class_name = box.get("label")
            if class_name not in class_mapping:
                continue  # 매핑에 없는 클래스는 무시

            class_id = class_mapping[class_name]
            xtl = float(box.get("xtl"))
            ytl = float(box.get("ytl"))
            xbr = float(box.get("xbr"))
            ybr = float(box.get("ybr"))

            # YOLO 포맷 좌표 변환
            x_center = ((xtl + xbr) / 2) / img_width
            y_center = ((ytl + ybr) / 2) / img_height
            bbox_width = (xbr - xtl) / img_width
            bbox_height = (ybr - ytl) / img_height

            yolo_annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}")

        # YOLO 포맷 TXT 저장
        output_txt_path = os.path.join(output_dir, f"{os.path.splitext(img_name)[0]}.txt")
        if yolo_annotations:
            with open(output_txt_path, "w") as f:
                f.write("\n".join(yolo_annotations))
            print(f"✅ 변환 완료: {output_txt_path}")
        else:
            print(f"⚠️ {img_name}: 객체 없음, 변환 생략")

def process_all_xml(input_dir, output_dir):
    """
    input_dir 내 모든 XML 파일을 YOLO 포맷으로 변환
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    xml_files = [f for f in os.listdir(input_dir) if f.endswith(".xml")]
    if not xml_files:
        print("❌ 변환할 XML 파일이 없습니다.")
        return

    for xml_file in xml_files:
        xml_path = os.path.join(input_dir, xml_file)
        convert_xml_to_yolo(xml_path, output_dir)

# 실행 예제 (모든 XML 변환)
xml_input_folder = "validation/labels"
output_directory = "validation/labels_yolo"

process_all_xml(xml_input_folder, output_directory)

In [ ]:
import yaml

# YOLO 학습을 위한 YAML 파일 생성 함수
def create_yaml_file(output_path, class_mapping):
    yaml_data = {
        "path": "data",  # 데이터셋 루트 경로
        "train": "train/images",  # 학습 데이터 경로
        "val": "validation/images",  # 검증 데이터 경로
        "nc": len(class_mapping),  # 클래스 개수 (자동 설정)
        "names": list(class_mapping.keys())  # 클래스 리스트
    }

    with open(output_path, "w") as f:
        yaml.dump(yaml_data, f, default_flow_style=False, allow_unicode=True)

    print(f"✅ YAML 파일 생성 완료: {output_path}")

# YAML 파일 생성 실행
yaml_output_path = "car.yaml"
create_yaml_file(yaml_output_path, class_mapping)

In [ ]:
from ultralytics import YOLO

# 1. 사전학습된 YOLO11x 모델 불러오기
model = YOLO("yolo11x.pt")  # YOLO11x 사전학습 모델 사용

# 2. 학습 실행
model.train(
    data="data.yaml",  # 학습 데이터셋 설정 파일
    epochs=100,  # 추가 학습 진행
    batch=16,  # 가능하면 32로 변경
    imgsz=640,
    workers=4,
    patience = 15,
    device="cuda:1",
    lr0=0.005,  # 학습률 감소 적용
    lrf=0.0001,  # 최종 학습률 조정
)

# 3. 검증 실행 (Validation)
metrics = model.val()
print("📊 검증 결과:", metrics)

In [ ]:
# data3000
from ultralytics import YOLO

# 1. 사전학습된 YOLO11x 모델 불러오기
model = YOLO("yolo11x.pt")  # YOLO11x 사전학습 모델 사용

# 2. 학습 실행
model.train(
    data="data3000/data3000.yaml",  # 학습 데이터셋 설정 파일
    epochs=100,  # 추가 학습 진행
    batch=16,  # 가능하면 32로 변경
    imgsz=640,
    workers=4,
    patience = 15,
    device="cuda:1",
    lr0=0.005,  # 학습률 감소 적용
    lrf=0.0001,  # 최종 학습률 조정
)

# 3. 검증 실행 (Validation)
metrics = model.val()
print("📊 검증 결과:", metrics)

In [ ]:
from ultralytics import YOLO

model = YOLO("runs/detect/train38/weights/best.pt")

results = model.predict(
    "for_test.jpg",
    save=False,
    imgsz=1280,
    conf=0.5,
    device="cuda:1",
)
for r in results:
    print(r.boxes)

In [ ]:
for r in results:
    boxes = r.boxes.xyxy
    
    for box in boxes:
        print(box)

In [ ]:
for r in results:
    image_path = r.path # 현재 이미지의 path
    boxes = r.boxes.xyxy # 현재 이미지의 bbox의 xy좌표값들
    cls = r.boxes.cls # 현재 이미지의 bbox의 class들
    conf = r.boxes.conf # 현재 이미지의 bbox의 conf값
    cls_dict = r.names # 지금 예제는 {0: 'joint', 1: 'side'}
    
    # boxes, cls, conf 개수는 같기 때문에 zip으로 한번 묶어준다
    for box, cls_number, conf in zip(boxes, cls, conf):
        conf_number = float(conf.item())
        cls_number_int = int(cls_number.item())
        cls_name = cls_dict[cls_number_int]
        x1, y1, x2, y2 = box
        x1_int = int(x1.item())
        y1_int = int(y1.item())
        x2_int = int(x2.item())
        y2_int = int(y2.item())
        print(x1_int, y1_int, x2_int, y2_int, cls_name)

In [ ]:
for r in results:
    image_path = r.path # 현재 이미지의 path
    boxes = r.boxes.xyxy # 현재 이미지의 bbox의 xy좌표값들
    cls = r.boxes.cls # 현재 이미지의 bbox의 class들
    conf = r.boxes.conf # 현재 이미지의 bbox의 conf값
    cls_dict = r.names # 지금 예제는 {0: 'joint', 1: 'side'}
    
    import cv2

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, c = image.shape

    image = cv2.resize(image, (640, 640)) # 출력할 이미지 사이즈 조정

    for box, cls_number, conf in zip(boxes, cls, conf):
        conf_number = float(conf.item())
        cls_number_int = int(cls_number.item())
        cls_name = cls_dict[cls_number_int]
        x1, y1, x2, y2 = box
        x1_int = int(x1.item())
        y1_int = int(y1.item())
        x2_int = int(x2.item())
        y2_int = int(y2.item())
        print(x1_int, y1_int, x2_int, y2_int, cls_name)

        # 출력할 이미지 사이즈를 조정했기 때문에 좌표값도 같이 조정 한다
        scale_factor_x = 640 / w
        scale_factor_y = 640 / h
        x1_scale = int(x1_int * scale_factor_x)
        y1_scale = int(y1_int * scale_factor_y)
        x2_scale = int(x2_int * scale_factor_x)
        y2_scale = int(y2_int * scale_factor_y)

        image = cv2.rectangle(
            image, (x1_scale, y1_scale), (x2_scale, y2_scale), (0, 225, 0), 6
        )

    cv2.imwrite("pred.jpg", image) # 이미지 저장
    cv2.imshow("Test", image)
    cv2.waitKey(0)